# MÃ HÓA BIẾN SỐ PHÂN LOẠI VỚI SCIKIT-LEARN

Như chúng ta đã biết trong phân tích dữ liệu có hai nhóm biến số chính là biến số định lượng và biến số phân loại. Cụ thể:
- Biến số định lượng (numerical variables):
    - Biến số liên tục (continuous variable)
    - Biến số rời rạc (discrete variable)
- Biến số phân loại (categorical variables):
    - Biến số hai giá trị (binary variable or dichotomous variable)   
    - Biến số phân loại có thứ bậc (ordinal variables)
    - Biến số phân loại không có thứ bậc (nominal variables)
   
Biến số phân loại hay có khi còn gọi là biến số định tính, mỗi giá biến số có từ hai đến nhiều giá trị và mỗi giá trị đại diện cho một nhóm nào đó. Loại biến số này thông thường được lưu trữ dưới dạng kiểu dữ liệu chuỗi chính vì vậy mà đa phần các thuật toán thống kê và học máy đều không thể xử lý được (ngoại trừ một số thuật toán dựa trên mô hình cây quyết định như Decision Tree, Random forest, XGBoot, Adaboot... và dựa trên tần suất Naive Bayes). Chính vì vậy nhà phân tích cần mã hóa các biến số phân loại này thành dạng số. Một số phần mềm thống kê khác đã được nhà phát triển tích hợp công đoàn này vào thuật toán. Dưới đây là một số phương pháp mã hóa biến số phân loại thông dụng.  
Cho tập dữ liệu mô phỏng:

In [1]:
import pandas as pd
import numpy as np

data= pd.read_csv('data/bmi-simulation_v2.csv', index_col= 0)
data.head()

,gioi_tinh,ton_giao,bmi,beo_phi,tt_dinh_duong
0,Nam,Khong,21.57,False,Binh_thuong
1,Nam,Cong_Giao,20.95,False,Binh_thuong
2,Nu,Khong,25.33,True,Beo_phi_do_1
3,Nu,Khong,21.04,False,Binh_thuong
4,Nam,Khong,21.44,False,Binh_thuong


## 1. Mã hóa biến số hai giá trị  

Biến số hai giá trị đang có kiểu dữ liệu dạng chuỗi cần chuyển thành kiểu dữ liệu dạng số có giá trị 0 và 1 hoặc True và False (trong python True = 1 và False = 0). Trong tập dữ liệu trên có hai biến số hai giá trị là gioi_tinh và beo_phi. Biến beo_phi đã được mã hóa tuy nhiên biến gioi_tinh chưa được mã hóa.

In [2]:
# Cách 1: Xóa ngẫu nhiên một trong 2 cột
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

gioi_tinh_OneHotEncoder= OneHotEncoder(drop= 'if_binary', sparse= False)
gioi_tinh_transform= ColumnTransformer([('gioi_tinh OneHotEncoder', gioi_tinh_OneHotEncoder, ['gioi_tinh'])], )
gioi_tinh_transformer= gioi_tinh_transform.fit(data)

new_col_name= gioi_tinh_transformer.transformers_[0][1].get_feature_names(['gioi_tinh'])[0]
data[new_col_name]= gioi_tinh_transformer.transform(data)
data.head()

,gioi_tinh,ton_giao,bmi,beo_phi,tt_dinh_duong,gioi_tinh_Nu
0,Nam,Khong,21.57,False,Binh_thuong,0.0
1,Nam,Cong_Giao,20.95,False,Binh_thuong,0.0
2,Nu,Khong,25.33,True,Beo_phi_do_1,1.0
3,Nu,Khong,21.04,False,Binh_thuong,1.0
4,Nam,Khong,21.44,False,Binh_thuong,0.0


In [3]:
# Cách 2: Chỉ định cột được giữ lại
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

gioi_tinh_OneHotEncoder2= OneHotEncoder(categories= [['Nam']],sparse= False, handle_unknown='ignore')
gioi_tinh_transform2= ColumnTransformer([('gioi_tinh OneHotEncoder', gioi_tinh_OneHotEncoder2, ['gioi_tinh'])], )
gioi_tinh_transformer2= gioi_tinh_transform2.fit(data)

new_col_name= gioi_tinh_transformer2.transformers_[0][1].get_feature_names(['gioi_tinh'])[0]
data[new_col_name]= gioi_tinh_transformer2.transform(data)
data.head()

,gioi_tinh,ton_giao,bmi,beo_phi,tt_dinh_duong,gioi_tinh_Nu,gioi_tinh_Nam
0,Nam,Khong,21.57,False,Binh_thuong,0.0,1.0
1,Nam,Cong_Giao,20.95,False,Binh_thuong,0.0,1.0
2,Nu,Khong,25.33,True,Beo_phi_do_1,1.0,0.0
3,Nu,Khong,21.04,False,Binh_thuong,1.0,0.0
4,Nam,Khong,21.44,False,Binh_thuong,0.0,1.0


Còn nhiều cách khác nữa, trên đây chỉ là hai ví dụ.

## 2. Mã hóa biến số phân loại có thứ bậc

Biến số phân loại có thứ bậc có các giá trị có thể được sắp xếp theo một thứ tự nhất định, mỗi giá trị tương ứng với một bậc. Chính vì vậy chúng ta có thể mã hóa biến số này về kiểu số nguyên sao cho mỗi bậc cách nhau một khoảng nhất định (thông thường là 1 đơn vị). Sau khi mã hóa biến số này trở thành kiểu biến số rời rạc.  
Không có một cách tự động nào để mã hóa biến số này chính vì vậy nhà phân tích cần gán giá trị tương ứng với mỗi bậc.  
Ví dụ: mã hóa biến số tình trạng dinh dưỡng gồm 4 giá trị: Thieu_can < Binh_thuong < Thua_can < Beo_phi_do_1 < Beo_phi_do_2 < Beo_phi_do_3

In [4]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

ttdd_OrdinalEncoder= OrdinalEncoder(categories= [['Thieu_can', 'Binh_thuong', 'Thua_can', 'Beo_phi_do_1', 'Beo_phi_do_2', 'Beo_phi_do_3']])
ttdd_transform= ColumnTransformer([('ttdd_OrdinalEncoder', ttdd_OrdinalEncoder, ['tt_dinh_duong'])])
ttdd_transformer= ttdd_transform.fit(data)
data['tt_dinh_duong_recode']= ttdd_transformer.transform(data)
data[['tt_dinh_duong','tt_dinh_duong_recode']].head()

,tt_dinh_duong,tt_dinh_duong_recode
0,Binh_thuong,1.0
1,Binh_thuong,1.0
2,Beo_phi_do_1,3.0
3,Binh_thuong,1.0
4,Binh_thuong,1.0


OrdinalEncoder cũng hỗ trợ auto gán giá trị cho các nhóm tuy nhiên chắc chắn rằng thứ tự sẽ không giống bản chất của biến số. Vì vậy cần truyền 1 list chứa các list thứ bậc của các nhóm.  
Ngoài ra còn có thể sử dụng FunctionTransformer tương tự làm tại bài mã hóa biến định lượng.

## 3. Mã hóa biến số phân loại không có thứ bậc

### 3.1. Mã hóa thành biến số giả / biến số thay thế   
Phương pháp này là phương pháp sử dụng phổ biến trong mã hóa biến số phân loại. Với mỗi nhóm của biến số phân loại tạo một biến số tương ứng có hai giá trị là 1/True thể hiện nhóm đó còn 0/False là không phải.  
Như vậy, với một biến số có 5 nhóm sẽ tạo thành 5 biến số giả mà trong mỗi mẫu chỉ có 1 biến nhận giá trị 1/True và 4 biến còn lại đều nhận giá trị 0/False. Chúng ta có thể xóa bỏ 1 biến số thì lúc này nhóm mà biến số được xóa thể hiện bởi 4 biến số nhận giá trị 0.

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ton_giao_OneHotEncoder= OneHotEncoder(sparse= False)
ton_giao_tranform= ColumnTransformer([('ton_giao_OneHot', ton_giao_OneHotEncoder, ['ton_giao'])])
ton_giao_tranformer= ton_giao_tranform.fit(data)
ton_giao_dummies= ton_giao_tranformer.transform(data)
new_colnames= ton_giao_tranformer.transformers_[0][1].get_feature_names(['ton_giao'])
ton_giao_dummies= pd.DataFrame(ton_giao_dummies, columns= new_colnames)
data= data.join(ton_giao_dummies)

In [6]:
data[['ton_giao'] + new_colnames.tolist()].head(10)

,ton_giao,ton_giao_Cao_Dai,ton_giao_Cong_Giao,ton_giao_Hoa_Hao,ton_giao_Hoi_Giao,ton_giao_Khong,ton_giao_Phat_Giao,ton_giao_Tin_Lanh
0,Khong,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Cong_Giao,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Khong,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Khong,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Khong,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,Khong,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,Khong,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,Khong,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,Phat_Giao,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,Khong,0.0,0.0,0.0,0.0,1.0,0.0,0.0


Trong một số trường hợp muốn gộp chung các nhóm nhỏ lại để tiện phân tích và diễn giải kết quả. Có thể thực hiện bằng cách khai báo các nhóm chính cho tham số categories trong OneHotEncoder, các nhóm muốn gộp không cần khai báo, khi đó các nhóm gộp chung sẽ đều có một kiểu thể hiện là toàn bộ giá trị 0 ở các biến giả.  
Ví dụ trong biến ton_giao muốn gộp các nhóm Hoi_Giao, Cao_Dai, Tin_Lanh, Hoa_Hao thành một nhóm thì không cần khai báo các nhóm này vào tham số categories.

In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ton_giao_OneHotEncoder_2= OneHotEncoder(categories=[['Khong', 'Cong_Giao', 'Phat_Giao']], sparse= False, handle_unknown= 'ignore')
ton_giao_tranform_2= ColumnTransformer([('ton_giao_OneHot', ton_giao_OneHotEncoder_2, ['ton_giao'])])
ton_giao_tranformer_2= ton_giao_tranform_2.fit(data)
ton_giao_dummies_2= ton_giao_tranformer_2.transform(data)
new_colnames_2= ton_giao_tranformer_2.transformers_[0][1].get_feature_names(['ton_giao'])
ton_giao_dummies_2= pd.DataFrame(ton_giao_dummies_2, columns= new_colnames_2)
data[['ton_giao']].join(ton_giao_dummies_2).head(20)

,ton_giao,ton_giao_Khong,ton_giao_Cong_Giao,ton_giao_Phat_Giao
0,Khong,1.0,0.0,0.0
1,Cong_Giao,0.0,1.0,0.0
2,Khong,1.0,0.0,0.0
3,Khong,1.0,0.0,0.0
4,Khong,1.0,0.0,0.0
5,Khong,1.0,0.0,0.0
6,Khong,1.0,0.0,0.0
7,Khong,1.0,0.0,0.0
8,Phat_Giao,0.0,0.0,1.0
9,Khong,1.0,0.0,0.0


### 3.2. Mã hóa theo tần suất

Trong Scikit-Learn chưa hỗ trợ phương pháp này tuy nhiên chung ta hoàn toàn có thể sử dụng FunctionTransformer để điều chỉnh theo ý muốn.

In [8]:
freq_table= data['ton_giao'].value_counts(normalize= True)
def assign_freq(x):
    global freq_table
    x= x.values
    ls_freq= []
    for i in x:
        ls_freq.append(freq_table[i[0]])
    return np.array(ls_freq).reshape(x.shape)

In [9]:
from sklearn.preprocessing import FunctionTransformer
freq_func_transformer= FunctionTransformer(func= assign_freq)
freq_tranform= ColumnTransformer([('freq_func_transform', freq_func_transformer, ['ton_giao'])])
freq_tranformer= freq_tranform.fit(data)
data['ton_giao_freq']= freq_tranformer.transform(data)
data[['ton_giao', 'ton_giao_freq']].head()

,ton_giao,ton_giao_freq
0,Khong,0.823333
1,Cong_Giao,0.080000
2,Khong,0.823333
3,Khong,0.823333
4,Khong,0.823333
